In [1]:
import numpy as np 
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
class Ising2D:
    def __init__(self,system,J,N,T,H,steps):
        #initialize the system variables 
        
        if system == "aligned":
            self.system = np.ones((N,N), dtype=int)      # N x N matrix of dipoles with spin up (1)
        elif system == "random":
            self.system =  2*np.random.randint(2, size = (N,N)) - 1     # N x N matrix of dipoles with spin up (1) or spin down (-1)
        else:
            print("Invalid system argument: input 'aligned' or 'random'")
            
        self.J      = J 
        self.N      = N           # Number of lattice points (dipoles) in the system
        self.T      = T           # Temperature of system
        self.H      = H           # External magnetic field
        self.steps  = steps       # Number of monte carlo steps 
        
        #initialize system parameters
        
        self.E_tot  = self.getTotalEnergy()
        self.M      = np.sum(self.system.astype(float))/(self.N**2)
        
    def getNeighbors(self,i,j):
        neighborhood =  self.system[(i+1)%self.N, j] + self.system[i,(j+1)%self.N] + self.system[(i-1)%self.N, j] + self.system[i,(j-1)%self.N]
        return neighborhood
        
    def getEnergy(self,i,j):
        s = self.system[i,j]
        neighborhood = self.getNeighbors(i,j)
        energy = ((-s * neighborhood) - (self.H * s))
        return energy
        
    def getTotalEnergy(self):
        energy = 0
        for i in range(self.N):
            for j in range(self.N):
                energy += self.getEnergy(i,j)
        return energy/2
        
        
    def getMagnetization(self):
        self.M = np.sum(self.system.astype(float))/(self.N**2)
    
    def metropolisStep(self,i,j):
        #calculate the contribution to energy at the specified lattice point before flipping spin
        #print(self.system)
        #print(self.E_tot)
        beta = 1/self.T
        E_i = self.getEnergy(i,j)
        #print('initial energy:', E_i)
        #flip the spin at position i,j
        self.system[i,j] *= -1
        #calculate the contribution to energy at specified lattice point after flipping spin
        E_f = self.getEnergy(i,j)
        #print('final energy:', E_f)
        dE = E_f - E_i
        #if the change in energy is negative, update the system energy and keep flipped spin. Do the same if Boltzman condition is satisfied. If not flip the spin back and do not update energy
        if dE < 0:
            #print('flip accepted')
            self.E_tot += dE
        elif np.random.rand() < np.exp(-beta*dE):
            #print('flip accepted by chance')
            self.E_tot += dE
        else:
            #print('flip rejected')
            self.system[i,j] *= -1
        self.getMagnetization()
            
    def run(self):
        for l in range(self.steps):
            i = np.random.randint(self.N)
            j = np.random.randint(self.N)
            self.metropolisStep(i,j)
    
    def test(self):
        M = []
        for l in range(self.steps):
            i = np.random.randint(self.N)
            j = np.random.randint(self.N)
            self.metropolisStep(i,j)
            M.append(self.M)
        return M
            
    def changingTemp(self,temps):
        a1 = np.linspace(0.0001, 2, 10)
        a2 = np.linspace(2, 3, 100)
        a3 = np.linspace(3,10,80)
        a4 = np.linspace(10,30,100)
        temp_array = np.concatenate((a1, a2,a3))
        temp_array_hot = np.concatenate((a1,a2,a3,a4))
        if temps == "cool":
            temps = temp_array.flip()
        elif temps == "heat":
            temps = temp_array
        elif temps == "hotcool":
            temps = temp_array_hot.fliplr()
        else:
            print("Error: invalid temps argument")
            return
        E = []
        M = []
        for i in np.nditer(temps):
            self.T = i
            self.run()
            E.append(self.E_tot)
            M.append(self.M)
        return E,M,temps
        
        
        
   

In [ ]:
N = 25
T = 10
J = 1
H = 0
steps = 2500


Ising = Ising2D('aligned',J,N,T,H,steps)
#Ising.run()

In [ ]:
E,M,temps = Ising.changingTemp('heat')

def mag(T):
    z = np.exp(-2/T)
    if T > 2.269185:
        return 0
    else:
        return (((1 + z**2)**(.25))*(1-6*z**2 + z**4)**(.125))/np.sqrt((1-z**2))

mags = []
for i in np.nditer(temps):
    mags.append(mag(i))

In [ ]:
plt.scatter(temps,np.array(E)/625, color = 'r')
#plt.scatter(temps,M, color = 'b')
#plt.plot(temps,mags)
plt.show()
        

In [ ]:
plt.scatter(temps,M, color = 'b')
plt.plot(temps,mags)
plt.show()

In [4]:
N = 25
T = 10
J = 1
H2 = 2
steps = 2500 

Ising2 = Ising2D('aligned',J,N,T,H2,steps)
E2,M2,temps = Ising2.changingTemp('heat')

NameError: name 'Ising' is not defined

In [ ]:
plt.scatter(temps,np.array(E2)/625, color = 'r')
#plt.scatter(temps,M, color = 'b')
#plt.plot(temps,mags)
plt.show()
        

In [ ]:
plt.scatter(temps,M2, color = 'b')
plt.show()

In [6]:
t = np.arange(10)
t = np.flip(t)
print(t)

[9 8 7 6 5 4 3 2 1 0]


In [7]:
a1 = np.linspace(0.0001, 2, 10)
a2 = np.linspace(2, 3, 100)
a3 = np.linspace(3,10,80)
a4 = np.linspace(10,30,100)
temp_array = np.concatenate((a1, a2,a3))
temp_array_hot = np.concatenate((a1,a2,a3,a4))
temps = np.flip(temp_array_hot)

In [8]:
temps

array([3.00000000e+01, 2.97979798e+01, 2.95959596e+01, 2.93939394e+01,
       2.91919192e+01, 2.89898990e+01, 2.87878788e+01, 2.85858586e+01,
       2.83838384e+01, 2.81818182e+01, 2.79797980e+01, 2.77777778e+01,
       2.75757576e+01, 2.73737374e+01, 2.71717172e+01, 2.69696970e+01,
       2.67676768e+01, 2.65656566e+01, 2.63636364e+01, 2.61616162e+01,
       2.59595960e+01, 2.57575758e+01, 2.55555556e+01, 2.53535354e+01,
       2.51515152e+01, 2.49494949e+01, 2.47474747e+01, 2.45454545e+01,
       2.43434343e+01, 2.41414141e+01, 2.39393939e+01, 2.37373737e+01,
       2.35353535e+01, 2.33333333e+01, 2.31313131e+01, 2.29292929e+01,
       2.27272727e+01, 2.25252525e+01, 2.23232323e+01, 2.21212121e+01,
       2.19191919e+01, 2.17171717e+01, 2.15151515e+01, 2.13131313e+01,
       2.11111111e+01, 2.09090909e+01, 2.07070707e+01, 2.05050505e+01,
       2.03030303e+01, 2.01010101e+01, 1.98989899e+01, 1.96969697e+01,
       1.94949495e+01, 1.92929293e+01, 1.90909091e+01, 1.88888889e+01,
      